In [33]:
from rdflib import Namespace, Graph, URIRef, Literal, XSD
import json

from get_schema_predicate import return_predicate
from flatten_dict import flatten_dictionary
from get_data_path import get_path

In [35]:
file = "yelp_academic_dataset_review.json"
file_path = get_path(file)

In [36]:
schema = Namespace("https://schema.org/")
example = Namespace("https://example.org/")

yelp_business = Namespace("https://www.yelp.com/biz/")
yelp_user = Namespace("https://www.yelp.com/user_details?userid=")
yelp_review_vc = Namespace("https://www.yelp.com/kg/review/votes_count/")

In [37]:
G = Graph()
G.bind(prefix="biz", namespace=yelp_business)

In [ ]:
with open(file=file_path, mode="r") as file:
    """
    The code is reading the json file line by line and then extracting the subject from the json file.
    Then it is deleting the subject from the json file and then flattening the json file.
    Then it is iterating over the flattened json file and adding the triples to the graph.
    If the object is of type None then it is not added to the graph.
    If the object is of type list then it is iterated over and added to the graph.
    If the object is of type string then it is added to the graph.
    """
    for line in file:
        try:
            line = json.loads(line)

            json_key = list(line.keys())[0] # Key of subject
            subject = line[json_key]
            del line[json_key]

            line = flatten_dictionary(line) # Flattens the nested dictionary

            for _predicate, _object in line.items():
                if isinstance(_object, type(None)) or str(_object).lower() == "none":
                    pass

                elif _predicate == "categories": # Categories are a string containing listed objects
                    obj_lst = _object.split(", ")

                    predicate, object_type = return_predicate(_predicate, _object)
                    for obj in obj_lst:
                        G.add(triple=(URIRef(yelp_business + subject), # Subject
                                  URIRef(predicate), # Predicate
                                  Literal(obj, datatype=object_type))) # Object

                else:
                    predicate, object_type = return_predicate(_predicate, _object)
                    G.add(triple=(URIRef(yelp_business + subject), # Subject
                                  URIRef(predicate), # Predicate
                                  Literal(_object, datatype=object_type))) # Object

        except Exception as e:
            print(e)

<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?


Error in SCHEMA!
cannot unpack non-iterable NoneType object
Error in SCHEMA!
cannot unpack non-iterable NoneType object


<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [ ]:
G.serialize(destination="kg.ttl")